# 분석 4: 뉴스 감성 영향 분석

## 목적
뉴스 긍정/부정이 집값에 미친 영향 분석

## 분석 대상
- **지역**: 강남구, 서초구, 송파구 (강남3구)
- **기간**: 2023년 ~ 2025년 (3년)
- **뉴스 출처**: 중앙일보 부동산 뉴스

## 분석 내용
- 감성지수 vs 실거래가 상관관계
- 긍정/부정 뉴스 증가 시점 전후 비교
- 시차 분석 (Lag Analysis)
- 구별 민감도 차이

## 핵심 질문
1. 긍정 뉴스가 많으면 집값이 오르는가?
2. 반응 속도는 얼마나 되는가?
3. 어느 구가 뉴스에 가장 민감한가?

## 1. 라이브러리 설치 및 임포트

In [1]:
# 필요한 라이브러리 설치
!pip install pandas numpy matplotlib seaborn plotly scipy requests beautifulsoup4 -q

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 통계
from scipy.stats import pearsonr, spearmanr, linregress

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


## 2. 기본 설정

In [3]:
# 크롤링 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}
BASE_URL = 'https://www.joongang.co.kr/realestate/news'

# 분석 대상
REGIONS = ['강남구', '서초구', '송파구']

# 강남 3구 키워드
DISTRICT_KEYWORDS = {
    '강남구': ['강남구', '강남역', '압구정', '청담', '삼성동', '대치동', '도곡동', '역삼동'],
    '서초구': ['서초구', '서초동', '반포', '잠원동', '방배동', '양재동'],
    '송파구': ['송파구', '잠실', '문정동', '가락동', '송파동', '석촌동', '방이동']
}

# 감성 분석용 키워드 (확장)
POSITIVE_KEYWORDS = [
    '상승', '급등', '호재', '인기', '완판', '신고가', '상한가', '활기', '회복', '기대',
    '활황', '강세', '호황', '투자', '수익', '개발', '재건축', '프리미엄', '청약', '당첨',
    '분양', '매수', '입주', 'GTX', '개통', '학군', '인프라'
]

NEGATIVE_KEYWORDS = [
    '하락', '급락', '악재', '침체', '미분양', '폭락', '불안', '위기', '규제', '충격',
    '거래절벽', '매물증가', '매도', '손실', '역전세', '깡통전세', '대출', '금리',
    '세금', '종부세', '양도세', '취득세', '버블', '과열', '조정', '경기침체'
]

print("✅ 설정 완료")
print(f"   긍정 키워드: {len(POSITIVE_KEYWORDS)}개")
print(f"   부정 키워드: {len(NEGATIVE_KEYWORDS)}개")

✅ 설정 완료
   긍정 키워드: 27개
   부정 키워드: 26개


## 3. 크롤링 함수 정의

In [4]:
def crwl_joongang_realestate(start_page=1, end_page=100):
    """중앙일보 부동산 섹션 크롤링"""
    all_news = []
    
    for page in range(start_page, end_page + 1):
        url = BASE_URL if page == 1 else f"{BASE_URL}?page={page}"
        print(f"크롤링 중: 페이지 {page}", end=" ")
        
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            if response.status_code != 200:
                print(f"[실패: {response.status_code}]")
                continue
            
            soup = BeautifulSoup(response.content, 'html.parser')
            cards = soup.select('li.card')
            
            if not cards:
                print("[종료: 기사 없음]")
                break
            
            print(f"[{len(cards)}건]")
            
            for card in cards:
                headline = card.select_one('h2.headline')
                date = card.select_one('p.date')
                link = card.select_one('a')
                desc = card.select_one('p.description, p.desc, span.desc')
                
                if headline:
                    all_news.append({
                        'title': headline.get_text(strip=True),
                        'date': date.get_text(strip=True) if date else '',
                        'url': link['href'] if link else '',
                        'content': desc.get_text(strip=True) if desc else ''
                    })
            
            time.sleep(0.5)
            
        except Exception as e:
            print(f"[에러: {e}]")
            continue
    
    print(f"\n✅ 총 {len(all_news)}개 수집 완료")
    return all_news

## 4. 데이터 처리 함수 정의

In [5]:
def parse_date(date_str):
    """날짜 문자열 파싱"""
    if not date_str:
        return None
    match = re.search(r'(\d{4})[.\-](\d{1,2})[.\-](\d{1,2})', date_str)
    if match:
        year, month, day = match.groups()
        return datetime(int(year), int(month), int(day))
    return None


def filter_by_date_range(news_list, start_date='2023-01-01', end_date='2025-12-31'):
    """날짜 범위로 필터링"""
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    
    filtered = []
    for news in news_list:
        parsed = parse_date(news['date'])
        if parsed and start <= parsed <= end:
            news['parsed_date'] = parsed
            news['year'] = parsed.year
            news['month'] = parsed.month
            news['year_month'] = parsed.strftime('%Y-%m')
            filtered.append(news)
    return filtered


def identify_district(text):
    """기사 텍스트에서 관련 구 식별"""
    districts = []
    for district, keywords in DISTRICT_KEYWORDS.items():
        if any(kw in text for kw in keywords):
            districts.append(district)
    return districts


def calculate_sentiment_score(text):
    """키워드 기반 감성 점수 계산 (-1 ~ 1)"""
    if not text:
        return 0, 0, 0
    
    pos_count = sum(1 for word in POSITIVE_KEYWORDS if word in text)
    neg_count = sum(1 for word in NEGATIVE_KEYWORDS if word in text)
    
    total = pos_count + neg_count
    if total == 0:
        return 0, 0, 0
    
    score = (pos_count - neg_count) / total
    return score, pos_count, neg_count


def process_news_data(news_list):
    """뉴스 데이터 처리: 구 식별 + 감성 분석"""
    result = []
    for news in news_list:
        text = news['title'] + ' ' + news.get('content', '')
        districts = identify_district(text)
        
        if districts:  # 강남3구 관련 기사만
            score, pos_count, neg_count = calculate_sentiment_score(text)
            
            news['region'] = districts[0]  # 첫 번째 구
            news['districts'] = ', '.join(districts)
            news['sentiment_score'] = score
            news['positive_count'] = pos_count
            news['negative_count'] = neg_count
            news['sentiment_label'] = '긍정' if score > 0.1 else ('부정' if score < -0.1 else '중립')
            
            result.append(news)
    return result


print("✅ 함수 정의 완료")

✅ 함수 정의 완료


## 5. 뉴스 크롤링 실행

In [6]:
# 크롤링 실행 (200페이지)
all_news = crwl_joongang_realestate(start_page=1, end_page=200)

크롤링 중: 페이지 1 [30건]
크롤링 중: 페이지 2 [29건]
크롤링 중: 페이지 3 [29건]
크롤링 중: 페이지 4 [29건]
크롤링 중: 페이지 5 [29건]
크롤링 중: 페이지 6 [29건]
크롤링 중: 페이지 7 [29건]
크롤링 중: 페이지 8 [29건]
크롤링 중: 페이지 9 [29건]
크롤링 중: 페이지 10 [29건]
크롤링 중: 페이지 11 [29건]
크롤링 중: 페이지 12 [29건]
크롤링 중: 페이지 13 [29건]
크롤링 중: 페이지 14 [29건]
크롤링 중: 페이지 15 [29건]
크롤링 중: 페이지 16 [29건]
크롤링 중: 페이지 17 [29건]
크롤링 중: 페이지 18 [29건]
크롤링 중: 페이지 19 [29건]
크롤링 중: 페이지 20 [29건]
크롤링 중: 페이지 21 [29건]
크롤링 중: 페이지 22 [29건]
크롤링 중: 페이지 23 [29건]
크롤링 중: 페이지 24 [29건]
크롤링 중: 페이지 25 [29건]
크롤링 중: 페이지 26 [29건]
크롤링 중: 페이지 27 [29건]
크롤링 중: 페이지 28 [29건]
크롤링 중: 페이지 29 [29건]
크롤링 중: 페이지 30 [29건]
크롤링 중: 페이지 31 [29건]
크롤링 중: 페이지 32 [29건]
크롤링 중: 페이지 33 [29건]
크롤링 중: 페이지 34 [29건]
크롤링 중: 페이지 35 [29건]
크롤링 중: 페이지 36 [29건]
크롤링 중: 페이지 37 [29건]
크롤링 중: 페이지 38 [29건]
크롤링 중: 페이지 39 [29건]
크롤링 중: 페이지 40 [29건]
크롤링 중: 페이지 41 [29건]
크롤링 중: 페이지 42 [29건]
크롤링 중: 페이지 43 [29건]
크롤링 중: 페이지 44 [29건]
크롤링 중: 페이지 45 [29건]
크롤링 중: 페이지 46 [29건]
크롤링 중: 페이지 47 [29건]
크롤링 중: 페이지 48 [29건]
크롤링 중: 페이지 49 [29건]
크롤링 중: 페이지 50 [29건]
크롤링 중: 페이

## 6. 데이터 필터링 및 감성 분석

In [7]:
# 2023-2025년 필터링
dated_news = filter_by_date_range(all_news, '2023-01-01', '2025-12-31')
print(f"📅 2023-2025년 기사: {len(dated_news)}개")

# 강남 3구 필터링 + 감성 분석
processed_news = process_news_data(dated_news)
print(f"🏢 강남 3구 관련: {len(processed_news)}개")

# DataFrame 변환
news_df = pd.DataFrame(processed_news)
news_df = news_df.sort_values('parsed_date', ascending=True).reset_index(drop=True)

print(f"\n✅ 최종 데이터: {len(news_df)}건")
news_df.head()

📅 2023-2025년 기사: 2640개
🏢 강남 3구 관련: 445개

✅ 최종 데이터: 445건


,title,date,url,content,parsed_date,year,month,year_month,region,districts,sentiment_score,positive_count,negative_count,sentiment_label
0,강남3구·용산 빼고 규제 확 푼다…“거래절벽 다소 완화될듯”,2023.01.02 19:19,https://www.joongang.co.kr/article/25130696,정부가 서울 강남 3구와 용산구만 남기고 부동산 규제지역을 모두 푼다. 여권 고위 ...,2023-01-02,2023,1,2023-01,송파구,송파구,-0.6,1,4,부정
1,"""강남 새 아파트 수난시대""…전셋값 폭락에 입주권도 10억 뚝",2023.01.03 17:05,https://www.joongang.co.kr/article/25130942,새 아파트에 입주할 수 있는 입주권 가격이 인근 기존 아파트보다 낮은 가격에 거래되...,2023-01-03,2023,1,2023-01,강남구,강남구,0.0,1,1,중립
2,강남·용산 외 부동산 규제 푼다,2023.01.03 00:18,https://www.joongang.co.kr/article/25130779,정부가 서울 강남 3구와 용산구만 남기고 부동산 규제지역을 모두 푼다. 여권 고위 ...,2023-01-03,2023,1,2023-01,송파구,송파구,-0.5,1,3,부정
3,서울서 4번째 ‘입지 끝판왕’…한강 780m 품은 신반포 2차유료 전용,2023.01.03 16:35,https://www.joongang.co.kr/article/25130929,한강 변에 위치한 서울 서초구 잠원동 신반포2차아파트의 김영일 재건축조합장에게 아파...,2023-01-03,2023,1,2023-01,서초구,서초구,1.0,1,0,긍정
4,"입주권 10억 뚝, 전셋값 하락…강남 신축 아파트도 꺾였다",2023.01.04 00:02,https://www.joongang.co.kr/article/25131030,새 아파트에 입주할 수 있는 입주권 가격이 인근 기존 아파트보다 낮은 가격에 거래되...,2023-01-04,2023,1,2023-01,강남구,강남구,0.0,1,1,중립


## 7. 뉴스 데이터 기본 정보 확인

In [8]:
print("=" * 60)
print("📰 뉴스 데이터 기본 정보")
print("=" * 60)

print(f"\n전체 뉴스: {len(news_df):,}건")
print(f"기간: {news_df['parsed_date'].min().strftime('%Y-%m-%d')} ~ {news_df['parsed_date'].max().strftime('%Y-%m-%d')}")

# 월별 평균 뉴스 건수
months_count = news_df['year_month'].nunique()
print(f"분석 기간: {months_count}개월")
print(f"월평균 뉴스: {len(news_df)/months_count:.0f}건")

# 구별 뉴스 건수
print(f"\n[구별 뉴스 건수]")
print(news_df['region'].value_counts())

# 감성 분포
print(f"\n[감성 분포]")
print(news_df['sentiment_label'].value_counts())

# 평균 감성 점수
print(f"\n평균 감성 점수: {news_df['sentiment_score'].mean():.4f}")

📰 뉴스 데이터 기본 정보

전체 뉴스: 445건
기간: 2023-01-02 ~ 2025-12-16
분석 기간: 36개월
월평균 뉴스: 12건

[구별 뉴스 건수]
region
강남구    224
서초구    133
송파구     88
Name: count, dtype: int64

[감성 분포]
sentiment_label
긍정    265
중립    102
부정     78
Name: count, dtype: int64

평균 감성 점수: 0.4163


## 8. 감성 키워드 빈도 분석

In [9]:
# 긍정 키워드 빈도
pos_counts = {}
for keyword in POSITIVE_KEYWORDS:
    count = news_df['title'].str.contains(keyword, na=False).sum()
    count += news_df['content'].str.contains(keyword, na=False).sum()
    if count > 0:
        pos_counts[keyword] = count

# 부정 키워드 빈도
neg_counts = {}
for keyword in NEGATIVE_KEYWORDS:
    count = news_df['title'].str.contains(keyword, na=False).sum()
    count += news_df['content'].str.contains(keyword, na=False).sum()
    if count > 0:
        neg_counts[keyword] = count

# 데이터프레임 변환
pos_df = pd.DataFrame(pos_counts.items(), columns=['keyword', 'count'])
pos_df = pos_df.sort_values('count', ascending=False).head(10)

neg_df = pd.DataFrame(neg_counts.items(), columns=['keyword', 'count'])
neg_df = neg_df.sort_values('count', ascending=False).head(10)

# 총합 비교
total_pos = sum(pos_counts.values()) if pos_counts else 0
total_neg = sum(neg_counts.values()) if neg_counts else 0

print(f"긍정 키워드 총 등장: {total_pos:,}회")
print(f"부정 키워드 총 등장: {total_neg:,}회")
print(f"긍정/부정 비율: {total_pos/(total_neg+1):.2f}")

긍정 키워드 총 등장: 767회
부정 키워드 총 등장: 281회
긍정/부정 비율: 2.72


In [10]:
# 시각화 - 긍정 키워드 TOP 10
if len(pos_df) > 0:
    fig_pos = px.bar(pos_df, x='keyword', y='count', title='📈 긍정 키워드 TOP 10',
                     color_discrete_sequence=['#4CAF50'], text='count')
    fig_pos.update_traces(textposition='outside')
    fig_pos.update_layout(template='plotly_white', height=400)
    fig_pos.show()

# 시각화 - 부정 키워드 TOP 10
if len(neg_df) > 0:
    fig_neg = px.bar(neg_df, x='keyword', y='count', title='📉 부정 키워드 TOP 10',
                     color_discrete_sequence=['#F44336'], text='count')
    fig_neg.update_traces(textposition='outside')
    fig_neg.update_layout(template='plotly_white', height=400)
    fig_neg.show()

## 9. 월별 감성지수 집계

In [11]:
# 전체 월별 감성지수
monthly_sentiment = news_df.groupby('year_month').agg(
    avg_sentiment=('sentiment_score', 'mean'),
    sentiment_std=('sentiment_score', 'std'),
    news_count=('title', 'count'),
    positive_count=('positive_count', 'sum'),
    negative_count=('negative_count', 'sum'),
    positive_ratio=('sentiment_label', lambda x: (x == '긍정').sum() / len(x) * 100)
).reset_index()

monthly_sentiment['date'] = pd.to_datetime(monthly_sentiment['year_month'] + '-01')

# 구별 월별 감성지수
regional_sentiment = news_df.groupby(['year_month', 'region']).agg(
    avg_sentiment=('sentiment_score', 'mean'),
    news_count=('title', 'count'),
    positive_ratio=('sentiment_label', lambda x: (x == '긍정').sum() / len(x) * 100)
).reset_index()

regional_sentiment['date'] = pd.to_datetime(regional_sentiment['year_month'] + '-01')

print("📅 월별 감성지수:")
monthly_sentiment.head(10)

📅 월별 감성지수:


,year_month,avg_sentiment,sentiment_std,news_count,positive_count,negative_count,positive_ratio,date
0,2023-01,-0.008333,0.735804,16,20,23,37.500000,2023-01-01
1,2023-02,0.071111,0.774378,15,28,19,46.666667,2023-02-01
2,2023-03,0.296296,0.748758,18,15,6,50.000000,2023-03-01
3,2023-04,0.626667,0.710555,10,15,4,80.000000,2023-04-01
4,2023-05,0.458333,0.469295,8,9,3,62.500000,2023-05-01
5,2023-06,0.400000,0.699206,10,13,1,50.000000,2023-06-01
6,2023-07,0.427083,0.680329,16,20,10,62.500000,2023-07-01
7,2023-08,0.100000,0.737865,10,4,5,30.000000,2023-08-01
8,2023-09,0.833333,0.389249,12,22,1,83.333333,2023-09-01
9,2023-10,0.733333,0.435890,9,13,1,77.777778,2023-10-01


## 10. 실거래가 데이터 로드 (샘플)

> **참고**: 실제 분석 시 팀에서 수집한 실거래가 데이터로 교체하세요.

In [12]:
# ============================================
# 실제 데이터 로드 (팀원 API 데이터)
# ============================================
# price_df = pd.read_csv('실거래가_데이터.csv')

# ============================================
# 샘플 데이터 생성 (테스트용)
# ============================================
np.random.seed(42)

months = pd.date_range(start='2023-01-01', end='2025-12-31', freq='M')
base_prices = {'강남구': 25, '서초구': 22, '송파구': 18}

price_data = []
for region in REGIONS:
    base = base_prices[region]
    for i, month in enumerate(months):
        price = base + (i * 0.08) + np.random.normal(0, 0.3)
        price_data.append({
            'date': month,
            'region': region,
            'avg_price': round(price, 2)
        })

price_df = pd.DataFrame(price_data)
price_df['year_month'] = price_df['date'].dt.strftime('%Y-%m')

print(f"📊 실거래가 데이터: {len(price_df):,}건")
print(f"기간: {price_df['date'].min().strftime('%Y-%m')} ~ {price_df['date'].max().strftime('%Y-%m')}")
price_df.head()

📊 실거래가 데이터: 108건
기간: 2023-01 ~ 2025-12


,date,region,avg_price,year_month
0,2023-01-31,강남구,25.15,2023-01
1,2023-02-28,강남구,25.04,2023-02
2,2023-03-31,강남구,25.35,2023-03
3,2023-04-30,강남구,25.70,2023-04
4,2023-05-31,강남구,25.25,2023-05


## 11. 감성지수 vs 집값 상관관계 분석

In [13]:
# 전체 평균 가격 (강남3구 평균)
monthly_price = price_df.groupby('year_month')['avg_price'].mean().reset_index()
monthly_price['date'] = pd.to_datetime(monthly_price['year_month'] + '-01')

# 감성지수와 가격 병합
merged_df = pd.merge(
    monthly_sentiment[['date', 'year_month', 'avg_sentiment', 'positive_ratio', 'news_count']],
    monthly_price[['date', 'avg_price']],
    on='date',
    how='inner'
)

# 상관계수 계산
if len(merged_df) > 5:
    corr_sentiment_price, p_value = pearsonr(merged_df['avg_sentiment'], merged_df['avg_price'])
else:
    corr_sentiment_price, p_value = 0, 1.0

print("=" * 50)
print("📊 감성지수 vs 집값 상관관계 분석")
print("=" * 50)
print(f"상관계수 (Pearson r): {corr_sentiment_price:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"통계적 유의성: {'✅ 유의함 (p < 0.05)' if p_value < 0.05 else '❌ 유의하지 않음'}")

📊 감성지수 vs 집값 상관관계 분석
상관계수 (Pearson r): 0.0066
P-value: 0.9695
통계적 유의성: ❌ 유의하지 않음


## 12. 시차 분석 (Lag Analysis)

뉴스 감성이 집값에 영향을 미치기까지 시간이 걸리는지 분석

In [14]:
def calculate_lag_correlation(df, sentiment_col, price_col, max_lag=6):
    """시차별 상관계수 계산"""
    results = []
    
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            sentiment = df[sentiment_col].iloc[-lag:].values
            price = df[price_col].iloc[:lag].values
        elif lag > 0:
            sentiment = df[sentiment_col].iloc[:-lag].values
            price = df[price_col].iloc[lag:].values
        else:
            sentiment = df[sentiment_col].values
            price = df[price_col].values
        
        if len(sentiment) > 5 and len(price) > 5:
            corr, p_val = pearsonr(sentiment, price)
            results.append({
                'lag': lag,
                'correlation': corr,
                'p_value': p_val,
                'significant': '✅' if p_val < 0.05 else ''
            })
    
    return pd.DataFrame(results)

# 시차 분석 실행
lag_results = calculate_lag_correlation(merged_df, 'avg_sentiment', 'avg_price', max_lag=6)

print("📊 시차별 상관계수:")
print("(양수 lag = 감성이 가격에 선행, 음수 lag = 가격이 감성에 선행)")
print(lag_results.to_string(index=False))

# 최적 시차 찾기
if not lag_results.empty:
    best_lag = lag_results.loc[lag_results['correlation'].abs().idxmax()]
    print(f"\n🎯 가장 강한 상관관계: {int(best_lag['lag'])}개월 시차 (r={best_lag['correlation']:.4f})")

📊 시차별 상관계수:
(양수 lag = 감성이 가격에 선행, 음수 lag = 가격이 감성에 선행)
 lag  correlation  p_value significant
  -6    -0.199544 0.290428            
  -5    -0.130814 0.483033            
  -4    -0.133534 0.466246            
  -3    -0.126910 0.481567            
  -2    -0.166768 0.345850            
  -1    -0.108926 0.533369            
   0     0.006606 0.969500            
   1     0.042466 0.808610            
   2     0.070149 0.693415            
   3     0.053921 0.765684            
   4    -0.036353 0.843415            
   5     0.007328 0.968793            
   6     0.321180 0.083521            

🎯 가장 강한 상관관계: 6개월 시차 (r=0.3212)


## 13. 구별 민감도 분석

In [15]:
# 구별 상관계수 계산
regional_correlation = []

for region in REGIONS:
    region_sent = regional_sentiment[regional_sentiment['region'] == region]
    region_price = price_df[price_df['region'] == region]
    
    # 날짜 매칭 문제 해결: year_month 기준으로 병합
    region_merged = pd.merge(
        region_sent[['year_month', 'avg_sentiment']],
        region_price[['year_month', 'avg_price']],
        on='year_month',
        how='inner'
    )
    
    if len(region_merged) >= 5:
        corr, p_val = pearsonr(region_merged['avg_sentiment'], region_merged['avg_price'])
        
        regional_correlation.append({
            'region': region,
            'correlation': corr,
            'p_value': p_val,
            'sensitivity': abs(corr),
            'significant': '✅' if p_val < 0.05 else ''
        })

sensitivity_df = pd.DataFrame(regional_correlation)

# 데이터가 없을 경우 처리 (KeyError 방지)
if not sensitivity_df.empty:
    sensitivity_df = sensitivity_df.sort_values('correlation', ascending=False)
else:
    # 데이터가 없을 경우 빈 DataFrame 생성 (컬럼 정의)
    sensitivity_df = pd.DataFrame(columns=['region', 'correlation', 'p_value', 'sensitivity', 'significant'])

print("📊 강남3구 뉴스 민감도 비교:")
print(sensitivity_df.to_string(index=False))

if len(sensitivity_df) > 0:
    print(f"\n🏆 가장 민감한 지역: {sensitivity_df.iloc[0]['region']}")
else:
    print("\n⚠️ 분석 가능한 충분한 데이터가 없습니다 (매칭된 데이터 부족).")

📊 강남3구 뉴스 민감도 비교:
region  correlation  p_value  sensitivity significant
   송파구     0.146956 0.422204     0.146956            
   강남구     0.066363 0.704860     0.066363            
   서초구    -0.019131 0.917228     0.019131            

🏆 가장 민감한 지역: 송파구


## 14. 구별 회귀분석

In [16]:
print("=" * 60)
print("📊 구별 감성-집값 회귀분석")
print("=" * 60)

regression_results = []

for region in REGIONS:
    region_sent = regional_sentiment[regional_sentiment['region'] == region]
    region_price = price_df[price_df['region'] == region]
    
    region_data = pd.merge(
        region_sent[['date', 'avg_sentiment']],
        region_price[['date', 'avg_price']],
        on='date',
        how='inner'
    )
    
    if len(region_data) > 5:
        slope, intercept, r_value, p_value, std_err = linregress(
            region_data['avg_sentiment'],
            region_data['avg_price']
        )
        
        regression_results.append({
            'region': region,
            'r_squared': r_value**2,
            'slope': slope,
            'p_value': p_value
        })
        
        print(f"\n{region}:")
        print(f"  R² (설명력): {r_value**2:.3f} ({r_value**2*100:.1f}%)")
        print(f"  기울기: {slope:.3f}")
        print(f"  P-value: {p_value:.4f}")
        print(f"  해석: 감성지수 1 증가 시 집값 {slope:.2f}억 {'증가' if slope > 0 else '감소'}")

regression_df = pd.DataFrame(regression_results)

📊 구별 감성-집값 회귀분석


## 15. 시각화

### 15-1. 이중축 차트 (감성지수 + 집값)

In [17]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['avg_sentiment'],
               name="감성지수", line=dict(color='#1976D2', width=2),
               hovertemplate='날짜: %{x}<br>감성지수: %{y:.3f}<extra></extra>'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['avg_price'],
               name="평균 집값 (억원)", line=dict(color='#D32F2F', width=2),
               hovertemplate='날짜: %{x}<br>집값: %{y:.2f}억<extra></extra>'),
    secondary_y=True
)

fig.update_layout(
    title={'text': f'뉴스 감성지수 vs 강남3구 집값 추이<br><sup>상관계수: {corr_sentiment_price:.3f}</sup>', 'x': 0.5},
    xaxis_title="날짜", hovermode='x unified', template='plotly_white',
    legend=dict(x=0.01, y=0.99), height=500
)

fig.update_yaxes(title_text="감성지수", secondary_y=False)
fig.update_yaxes(title_text="집값 (억원)", secondary_y=True)

fig.show()

### 15-2. 긍정/부정 뉴스 비교

In [18]:
fig2 = go.Figure()

fig2.add_trace(go.Bar(x=monthly_sentiment['date'], y=monthly_sentiment['positive_count'],
                      name='긍정 키워드', marker_color='#4CAF50'))
fig2.add_trace(go.Bar(x=monthly_sentiment['date'], y=monthly_sentiment['negative_count'],
                      name='부정 키워드', marker_color='#F44336'))

fig2.update_layout(
    title='월별 긍정/부정 뉴스 키워드 비교',
    xaxis=dict(title='년월', tickangle=-45),
    yaxis=dict(title='키워드 빈도'),
    barmode='group', hovermode='x unified', template='plotly_white', height=500
)

fig2.show()

### 15-3. 시차 상관분석 시각화

In [19]:
fig3 = go.Figure()

colors = ['#4CAF50' if r > 0 else '#F44336' for r in lag_results['correlation']]
fig3.add_trace(
    go.Bar(x=lag_results['lag'], y=lag_results['correlation'],
           marker_color=colors, text=[f'{r:.3f}' for r in lag_results['correlation']],
           textposition='outside', hovertemplate='시차: %{x}개월<br>상관계수: %{y:.4f}<extra></extra>')
)

fig3.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)

fig3.update_layout(
    title={'text': '시차별 상관계수 (감성지수 → 집값)<br><sup>양수 시차 = 감성이 집값에 선행</sup>', 'x': 0.5},
    xaxis_title='시차 (개월)', yaxis_title='상관계수',
    template='plotly_white', xaxis=dict(tickmode='linear', tick0=-6, dtick=1), height=450
)

fig3.show()

### 15-4. 구별 감성 분포 (박스플롯)

In [20]:
fig4 = px.box(
    news_df, x='region', y='sentiment_score', color='region',
    title='구별 감성지수 분포 (민감도 비교)',
    labels={'region': '구', 'sentiment_score': '감성지수'},
    color_discrete_map={'강남구': '#FF6B6B', '서초구': '#4ECDC4', '송파구': '#95E1D3'}
)
fig4.update_layout(template='plotly_white', height=500)
fig4.show()

### 15-5. 구별 월별 감성 추이

In [21]:
fig5 = px.line(
    regional_sentiment, x='date', y='avg_sentiment', color='region',
    title='구별 월별 감성지수 추이', markers=True,
    labels={'date': '년월', 'avg_sentiment': '평균 감성지수', 'region': '구'},
    color_discrete_map={'강남구': '#FF6B6B', '서초구': '#4ECDC4', '송파구': '#95E1D3'}
)
fig5.update_layout(template='plotly_white', xaxis=dict(tickangle=-45), hovermode='x unified', height=500)
fig5.show()

### 15-6. 히트맵 (월별 × 구별 감성)

In [22]:
heatmap_data = regional_sentiment.pivot(index='region', columns='year_month', values='avg_sentiment').fillna(0)

fig6 = px.imshow(
    heatmap_data, title='월별 × 구별 평균 감성지수 히트맵',
    labels=dict(x='년월', y='구', color='감성지수'),
    color_continuous_scale='RdYlGn', aspect='auto'
)
fig6.update_layout(template='plotly_white', height=400)
fig6.show()

## 16. 통합 대시보드

In [23]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('① 감성지수 vs 집값 추이', '② 시차별 상관계수', '③ 월별 감성 분포', '④ 구별 민감도'),
    specs=[[{"secondary_y": True}, {}], [{}, {}]],
    vertical_spacing=0.15, horizontal_spacing=0.1
)

# ① 이중축 차트
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['avg_sentiment'], name='감성지수', line=dict(color='#1976D2')),
              row=1, col=1, secondary_y=False)
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['avg_price'], name='집값', line=dict(color='#D32F2F')),
              row=1, col=1, secondary_y=True)

# ② 시차 상관계수
lag_colors = ['#4CAF50' if r > 0 else '#F44336' for r in lag_results['correlation']]
fig.add_trace(go.Bar(x=lag_results['lag'], y=lag_results['correlation'], marker_color=lag_colors, showlegend=False),
              row=1, col=2)

# ③ 긍정/부정 비율
fig.add_trace(go.Scatter(x=monthly_sentiment['date'], y=monthly_sentiment['positive_ratio'], name='긍정비율(%)', line=dict(color='#4CAF50')),
              row=2, col=1)

# ④ 구별 민감도
if len(sensitivity_df) > 0:
    sens_colors = ['#4CAF50' if c > 0 else '#F44336' for c in sensitivity_df['correlation']]
    fig.add_trace(go.Bar(x=sensitivity_df['region'], y=sensitivity_df['sensitivity'], marker_color=sens_colors, showlegend=False),
                  row=2, col=2)

fig.update_layout(
    height=700, title_text='📊 뉴스 감성 영향 분석 대시보드',
    template='plotly_white', showlegend=True, legend=dict(x=1.02, y=1)
)

fig.show()

## 17. 최종 분석 결과 요약

In [24]:
print("=" * 70)
print("📊 뉴스 감성 영향 분석 최종 결과")
print("=" * 70)

print(f"\n[1] 데이터 수집 결과")
print(f"   - 분석 기사: {len(news_df):,}건")
print(f"   - 분석 기간: {news_df['parsed_date'].min().strftime('%Y-%m-%d')} ~ {news_df['parsed_date'].max().strftime('%Y-%m-%d')}")

print(f"\n[2] 감성 분석 결과")
print(f"   - 긍정 뉴스: {(news_df['sentiment_label']=='긍정').sum():,}건 ({(news_df['sentiment_label']=='긍정').mean()*100:.1f}%)")
print(f"   - 부정 뉴스: {(news_df['sentiment_label']=='부정').sum():,}건 ({(news_df['sentiment_label']=='부정').mean()*100:.1f}%)")
print(f"   - 평균 감성지수: {news_df['sentiment_score'].mean():.4f}")

print(f"\n[3] 상관관계 분석")
print(f"   - 전체 상관계수: {corr_sentiment_price:.4f}")
print(f"   - 통계적 유의성: {'✅ 유의함' if p_value < 0.05 else '❌ 유의하지 않음'}")

print(f"\n[4] 시차 분석")
print(f"   - 최적 반응 시차: {int(best_lag['lag'])}개월")
print(f"   - 해당 시차 상관계수: {best_lag['correlation']:.4f}")

print(f"\n[5] 구별 민감도 순위")
for i, row in sensitivity_df.iterrows():
    print(f"   {i+1}위: {row['region']} (r={row['correlation']:.4f})")

print("\n" + "=" * 70)

📊 뉴스 감성 영향 분석 최종 결과

[1] 데이터 수집 결과
   - 분석 기사: 445건
   - 분석 기간: 2023-01-02 ~ 2025-12-16

[2] 감성 분석 결과
   - 긍정 뉴스: 265건 (59.6%)
   - 부정 뉴스: 78건 (17.5%)
   - 평균 감성지수: 0.4163

[3] 상관관계 분석
   - 전체 상관계수: 0.0066
   - 통계적 유의성: ❌ 유의하지 않음

[4] 시차 분석
   - 최적 반응 시차: 6개월
   - 해당 시차 상관계수: 0.3212

[5] 구별 민감도 순위
   3위: 송파구 (r=0.1470)
   1위: 강남구 (r=0.0664)
   2위: 서초구 (r=-0.0191)



## 18. CSV 저장

In [26]:
# 저장 폴더 생성
import os
output_path = './analysis_results/'
os.makedirs(output_path, exist_ok=True)

# 뉴스 데이터 저장
news_df.to_csv(f'{output_path}news_with_sentiment.csv', index=False, encoding='utf-8-sig')

# 월별 감성지수 저장
monthly_sentiment.to_csv(f'{output_path}monthly_sentiment.csv', index=False, encoding='utf-8-sig')

# 구별 민감도 저장
sensitivity_df.to_csv(f'{output_path}regional_sensitivity.csv', index=False, encoding='utf-8-sig')

# 시차 분석 결과 저장
lag_results.to_csv(f'{output_path}lag_analysis.csv', index=False, encoding='utf-8-sig')

# 회귀분석 결과 저장
if len(regression_df) > 0:
    regression_df.to_csv(f'{output_path}regression_results.csv', index=False, encoding='utf-8-sig')

print("✅ 분석 결과가 저장되었습니다!")
print(f"   저장 위치: {output_path}")
print(f"\n   저장된 파일:")
print(f"   - news_with_sentiment.csv (뉴스 감성 결과)")
print(f"   - monthly_sentiment.csv (월별 감성지수)")
print(f"   - regional_sensitivity.csv (구별 민감도)")
print(f"   - lag_analysis.csv (시차 분석 결과)")
print(f"   - regression_results.csv (회귀분석 결과)")

✅ 분석 결과가 저장되었습니다!
   저장 위치: ./analysis_results/

   저장된 파일:
   - news_with_sentiment.csv (뉴스 감성 결과)
   - monthly_sentiment.csv (월별 감성지수)
   - regional_sensitivity.csv (구별 민감도)
   - lag_analysis.csv (시차 분석 결과)
   - regression_results.csv (회귀분석 결과)
